In [202]:
import re

In [203]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [204]:
with open('urine17.txt', 'r') as f:
    text = f.read()

In [143]:
text

'.   度\n\n项目\n检浑检浑量分析一 F微镜镜检\n路 肌\n\n检项目               结果单位参考区间/值检项目              结果单位   参考区间/值\n胆红素(BIL)             阴性(-)              阴性病理管型               0.4/ul          0-0.5\n酮体(KET)             阴性(-)             阴性结晶                0.7一Aul          0-10\n比重(S6)               1.022       1.003-1.030细菌              204.3/ul     0.0-1000.0\n尿糖(GLU)         阴性(-)         阴性每高倍视野细菌数     2.0105/ml\n尿潜血(BLD)           阴性(-)             阴性酵母             0.0Aul         0-0.1\n蛋白质(PRO)            阴性(-)             阴性粘液                0.7一/ul\n尿胆原(UBG)                  正常                阴性                     0.0Aul\npH值(PH)               6.0         4.5-8.0电导率             28.1mS/cm\n\n亚硝酸盐(NIT)       阴性(-)         阴性电导信息         4级\n\n尿白细胞(LEU)           阴性()              阴性镜检红细胞(RBC)         0-2/HP           0-3\n颜色                       黄色                     镜检白细胞                t+/HNP             0-5\n清晰度                            清晰                          镜检鳞状上皮细胞             P\n\

In [239]:
urine_dict = {}
urine_dict['尿胆原'] = ['尿胆原','尿胆原\(URO\)','尿胆原\(UBG\)','尿胆原\s*URO']
urine_dict['胆红素'] = ['胆红素','尿胆红素','胆红素\(BIL\)','胆红素\s*BIL']
urine_dict['酮体'] = ['酮体','尿酮体','酮体 \(KET\)','酮 体','酮体\(KET\)','酮体\s*KET']
urine_dict['隐血'] = ['血','隐血','潜血','尿隐血','尿潜血','隐血或红细胞\(BLD\)','尿潜血\(BLD\)','隐血\s*BLD']
urine_dict['尿蛋白'] = ['蛋白质','尿蛋白','蛋白质\(PRO\)','蛋白','蛋白质\s*PRO']
urine_dict['亚硝酸盐'] = ['亚硝酸盐','尿亚硝酸盐','亚硝酸盐\(NIT\)','亚硝酸\s*NIT']
urine_dict['葡萄糖'] = ['葡萄糖','尿糖','尿葡萄糖','尿糖\(GLU\)','葡萄糖\s*GLU']
urine_dict['比重'] = ['比重','尿比重','比重 \(SG\)','比 重','比重\(SG\)']
urine_dict['酸碱度'] = ['酸碱度','尿PH值','尿酸碱度','PH','酸碱度\(PH\)','尿PH','pH值\(PH\)','酸碱度\(pH\)','酸碱度\s*PH']
urine_dict['颜色'] = ['颜色','颜色 \(Color\)','尿液外观']
urine_dict['浑浊度'] = ['浊度','浑浊度','透明度\(TMD\)','透明度','尿透明度','外观','清晰度']
urine_dict['电导率'] = ['电导率','电导率\s*Cond']
urine_dict['电导率信息'] = ['电导率分级','电导率信息']
urine_dict['类酵母菌'] = ['类酵母菌','类酵母细胞数量','类酵母细胞标记','酵母样菌','类酵母细胞数量\s*YLC']
urine_dict['粘液丝'] = ['粘液丝','粘液丝数量','粘液','粘液丝数量\s*MCLS']
urine_dict['细菌'] = ['细菌计数','细菌','细菌\s*BACT']
urine_dict['维生素C'] = ['VC','维生素C','维生素C\s*VC']
urine_dict['完整红细胞比率'] = ['完整红细胞比率','完整红细胞百分比','完整红细胞百分比\s*NL-RBC%']
urine_dict['尿路感染信息'] = ['尿路感染信息']
urine_dict['尿钙'] = ['尿钙']
urine_dict['精子'] = ['精子']
urine_dict['完整红细胞绝对值'] = ['完整红细胞绝对值','完整红细胞绝对值\s*NL-RBC#']
urine_dict['其他有形成份'] = ['其他有形成份']
urine_dict['70%红细胞平均体积'] = ['70%红细胞平均体积','70%红细胞平均体积\s*RBC-P70']
urine_dict['白细胞(高倍视野)'] = ['白细胞\(HPF\)','每高倍视野白细胞数','白细胞\(高倍视野\)']
urine_dict['红细胞(高倍视野)'] = ['红细胞\(高倍\)','每高倍视野白细胞数','红细胞\(高倍视野\)']
urine_dict['上皮细胞(高倍视野)'] = ['上皮细胞\(高倍视野\)','上皮细胞\(高倍\)','每高倍视野上皮细胞数']
urine_dict['红细胞信息'] = ['红细胞信息','红细胞形态信息','红细胞形态变异信息','红细胞信息\s*info']
urine_dict['红细胞镜检'] = ['直接镜检-红细胞','镜检红细胞','红细胞\(镜检\)','镜检：红细胞\(RBC\)','红细胞镜检']
urine_dict['白细胞镜检'] = ['直接镜检-白细胞','镜检白细胞','白细胞\(镜检\)','镜检：白细胞','白细胞镜检']
urine_dict['上皮细胞镜检'] = ['上皮细胞\(镜检\)','上皮细胞镜检']
urine_dict['小圆上皮细胞'] = ['小圆上皮','小圆上皮细胞','小圆上皮细胞标记','小圆上皮细胞数量','小圆细胞']
urine_dict['管型'] = ['管型','管型计数','管型计数\(CAST\)','管型\s*(LPF)']
# sort the names according to their length, longest name first
for k, names in urine_dict.items():
    names.sort(key = lambda s: -len(s))

In [206]:
urine_dict

{'70%红细胞平均体积': ['70%红细胞平均体积\\s*RBC-P70', '70%红细胞平均体积'],
 '上皮细胞(高倍视野)': ['上皮细胞\\(高倍视野)', '上皮细胞\\(高倍\\)', '每高倍视野上皮细胞数'],
 '亚硝酸盐': ['亚硝酸盐\\(NIT\\)', '亚硝酸\\s*NIT', '尿亚硝酸盐', '亚硝酸盐'],
 '其他有形成份': ['其他有形成份'],
 '完整红细胞比率': ['完整红细胞百分比\\s*NL-RBC%', '完整红细胞百分比', '完整红细胞比率'],
 '完整红细胞绝对值': ['完整红细胞绝对值\\s*NL-RBC#', '完整红细胞绝对值'],
 '尿胆原': ['尿胆原\\(URO\\)', '尿胆原\\(UBG\\)', '尿胆原\\s*URO', '尿胆原'],
 '尿蛋白': ['蛋白质\\(PRO\\)', '蛋白质\\s*PRO', '蛋白质', '尿蛋白', '蛋白'],
 '尿路感染信息': ['尿路感染信息'],
 '尿钙': ['尿钙'],
 '比重': ['比重 \\(SG\\)', '比重\\(SG\\)', '尿比重', '比 重', '比重'],
 '浑浊度': ['透明度\\(TMD\\)', '尿透明度', '浑浊度', '透明度', '清晰度', '浊度', '外观'],
 '电导率': ['电导率\\s*Cond.', '电导率'],
 '电导率信息': ['电导率分级', '电导率信息'],
 '白细胞(高倍视野)': ['白细胞\\(高倍视野\\)', '白细胞\\(HPF\\)', '每高倍视野白细胞数'],
 '类酵母菌': ['类酵母细胞数量\\s*YLC', '类酵母细胞数量', '类酵母细胞标记', '类酵母菌', '酵母样菌'],
 '粘液丝': ['粘液丝数量\\s*MCLS', '粘液丝数量', '粘液丝', '粘液'],
 '精子': ['精子'],
 '红细胞(高倍视野)': ['红细胞\\(高倍视野\\)', '红细胞\\(高倍\\)', '每高倍视野白细胞数'],
 '红细胞信息': ['红细胞信息\\s*info', '红细胞形态变异信息', '红细胞形态信息', '红细胞信息'],
 '细菌': ['细菌\\s*BACT', '细

In [207]:
urine_tests = ['尿胆原','胆红素','酮体','隐血','尿蛋白','亚硝酸盐','葡萄糖','比重','酸碱度','颜色','浑浊度',
              '电导率','电导率信息','类酵母菌','粘液丝','细菌','维生素C','完整红细胞比率','尿路感染信息',
              '尿钙','精子','完整红细胞绝对值','其他有形成份','70%红细胞平均体积','白细胞(高倍视野)',
              '红细胞(高倍视野)','上皮细胞（高倍视野）','红细胞信息','红细胞镜检','白细胞镜检','上皮细胞镜检',
              '小圆上皮细胞']

In [240]:
# result is the dict where we store test names and values
result = {}
for test in urine_tests:
    result.setdefault(test, '')

In [209]:
#split the text as a list of strings
text_split = text.split()

In [210]:
text_split

['NO一s白',
 '一',
 '结果',
 '单位',
 '参考值',
 'N0',
 '项目',
 '结果',
 '单位',
 '1红细胞.',
 'RBC.8336.50f/ul0.00一25.0016白细胞',
 'WBC500(+3)',
 '2白细(UF)',
 'WBC(UF)9518.30f/ul0.00一30.0017酮体',
 'KET-0.5()',
 '3一皮细胞.',
 'EC',
 '233.201/ul0.00一9.00',
 '18亚硝酸',
 'NIT性(-)',
 '4小圆上皮细胞数量SRC',
 '2.90/ul',
 '19尿胆原',
 'UR0O',
 'Normal',
 '5管型.',
 'CAST',
 '0.24Aul0.00一1.0020胆红素',
 'BTL',
 '0(-)',
 '6病理管型',
 'P.ChAST1565.24/ul',
 '21蛋白质',
 'PRO3.0(+3)',
 '7细葡',
 'BACT13.40/ul0.00一3395.0022尿比重',
 'SC',
 '1.015',
 '8结数量',
 'XTAL',
 '0.00/ul',
 '23葡萄',
 'GLU.',
 '0(-)',
 '9粘液丝数量',
 'MCLS',
 '0.00/ul',
 '24隐血',
 'BLD',
 '80(+2)',
 '10类酶母细胞数量YLC',
 '0.00/ul',
 '25酸碱度',
 'PH',
 '7.50',
 'L1电导率',
 'Cond9.00msS/cm3.00一39.0026维生素C',
 'VC',
 '0(-)mmol/L',
 '12705细胞平均体积RBC-P70105.00',
 '27尿液外观',
 '红色',
 '13完整红细胞百分比NL-RBC100.00',
 '28透明度.',
 '浑浊',
 '14红细胞维值',
 'NL-RBC8336.50',
 '49',
 '检白细胞.',
 '+-',
 'HP',
 '15红细胞信息',
 'info',
 '均一型',
 '30镜检红细胞',
 '+++一',
 '胆']

In [241]:
# find the perfect match for test names
for k, names in urine_dict.items():
    for name in names:
        value = re.findall(''.join([name, '(\s*)(\S+)']), text)
        if value:
            result[k] = value[0][1]
            break

In [242]:
result

{'70%红细胞平均体积': '',
 '上皮细胞（高倍视野）': '',
 '亚硝酸盐': '性(-)',
 '其他有形成份': '',
 '完整红细胞比率': 'NL-RBC100.00',
 '完整红细胞绝对值': '',
 '尿胆原': 'UR0O',
 '尿蛋白': '3.0(+3)',
 '尿路感染信息': '',
 '尿钙': '',
 '比重': 'SC',
 '浑浊度': '.',
 '电导率': '9.00msS/cm3.00一39.0026维生素C',
 '电导率信息': '',
 '白细胞(高倍视野)': '',
 '类酵母菌': '',
 '粘液丝': '0.00/ul',
 '精子': '',
 '红细胞(高倍视野)': '',
 '红细胞信息': '均一型',
 '细菌': '',
 '维生素C': '0(-)mmol/L',
 '胆红素': 'BTL',
 '葡萄糖': '',
 '酮体': '-0.5()',
 '酸碱度': '7.50',
 '隐血': '80(+2)',
 '颜色': '红色'}

In [230]:
tmp = ['(UBG)123']
process.extract('\(UBG\)', tmp, limit=1)

[('(UBG)123', 90)]

In [234]:
for k, v in result.items():
    if not v:
        fuzzy = process.extract(k, text_split, limit=1)
        print(k, fuzzy)
        if fuzzy[0][1] > 60:
            name = fuzzy[0][0]
            if '(' in name:
                    index = name.find('(')
                    name = name[:index] + '\\' + name[index:]
            if ')' in name:
                    index = name.find(')')
                    name = name[:index] + '\\' + name[index:]
            value = re.findall(''.join([name, '(\s*)(\S+)']), text)
            if value:
                result[k] = value[0][1]

葡萄糖 [('23葡萄', 57)]
电导率信息 [('L1电导率', 60)]
类酵母菌 [('10类酶母细胞数量YLC', 45)]
细菌 [('1红细胞.', 45)]
尿路感染信息 [('15红细胞信息', 31)]
尿钙 [('19尿胆原', 45)]
精子 [('NO一s白', 0)]
完整红细胞绝对值 [('1红细胞.', 68)]
其他有形成份 [('NO一s白', 0)]
70%红细胞平均体积 [('12705细胞平均体积RBC-P70105.00', 72)]
白细胞(高倍视野) [('检白细胞.', 68)]
红细胞(高倍视野) [('1红细胞.', 68)]
上皮细胞（高倍视野） [('3一皮细胞.', 54)]


In [222]:
result

{'70%红细胞平均体积': '27尿液外观',
 '上皮细胞（高倍视野）': '',
 '亚硝酸盐': '性(-)',
 '其他有形成份': '',
 '完整红细胞比率': 'NL-RBC100.00',
 '完整红细胞绝对值': 'RBC.8336.50f/ul0.00一25.0016白细胞',
 '尿胆原': 'UR0O',
 '尿蛋白': '3.0(+3)',
 '尿路感染信息': '',
 '尿钙': '',
 '比重': 'SC',
 '浑浊度': '.',
 '电导率': '9.00msS/cm3.00一39.0026维生素C',
 '电导率信息': '',
 '白细胞(高倍视野)': '+-',
 '类酵母菌': '',
 '粘液丝': '0.00/ul',
 '精子': '',
 '红细胞(高倍视野)': 'RBC.8336.50f/ul0.00一25.0016白细胞',
 '红细胞信息': '均一型',
 '细菌': '',
 '维生素C': '0(-)mmol/L',
 '胆红素': 'BTL',
 '葡萄糖': '',
 '酮体': '-0.5()',
 '酸碱度': '7.50',
 '隐血': '80(+2)',
 '颜色': '红色'}